In [1]:
from __init__ import *

In [2]:
import natasha
tqdm.pandas()

In [3]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
geolocator = Nominatim(user_agent="my test app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.3)

In [4]:
with sqlite3.connect('data/avito_real_estate.db') as db:
    df = pd.read_sql('select * from moskva_RE',con=db)

In [5]:
doc = natasha.Doc(df['text'].str.lower().str.extract('([А-я]+ ремонт)').fillna('').apply('|'.join)[0])

In [6]:
segmenter = natasha.Segmenter()
morph_vocab = natasha.MorphVocab()
emb = natasha.NewsEmbedding()
morph_tagger = natasha.NewsMorphTagger(emb)

In [7]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [8]:
for token in doc.tokens:
     token.lemmatize(morph_vocab)

In [9]:
df['price'] = df['price'].astype(int)

In [10]:
df['rooms'] = df['title'].str.split(', ',expand=True)[0].str.extract('(\d)').astype(int)
df['m2']=df['title'].str.split(', ',expand=True)[1].str.replace(',','.').str.extract('(\d.*\d)').astype(float)
df[['floor','max_floor']]=df['title'].str.extract('(\d+/\d+)')[0].str.split('/',expand=True)
df['has_furniture']=df['text'].str.contains('мебель')
df['JK']=df['text'].str.contains('ЖК')
df['n_words'] = df['text'].str.count(' ')
df['repair_type'] = ' '.join([token.lemma for token in doc.tokens if token.lemma != 'ремонт']).split('|')
df['metro_name']=df['metro'].str.split(', ',expand=True)[0]
df['metro_dist']=df['metro'].str.split(', ',expand=True)[1].str.replace(',','.').str.extract('(\d+.\d?)').astype('float')[0].apply(lambda x: x*1000 if x<10 else x)
df['rubm2'] = (df['price'] / df['m2']).round()
df['room_size']= (df['m2'] / df['rooms']).round()
df['street']=df['adress'].str.split(', (\d.*)',expand=True)[0]
df['build']=df['adress'].str.split(', (\d.*)',expand=True)[1]

In [12]:
mid = geolocator.geocode("Moscow")

In [16]:
curr_rent=geocode(f'Москва, Союзный проспект 20')

In [17]:
mid

Location(Москва, Центральный федеральный округ, Россия, (55.7504461, 37.6174943, 0.0))

In [18]:
curr_rent

Location(20 к3, Союзный проспект, район Новогиреево, Москва, Центральный федеральный округ, 111396, Россия, (55.7474484, 37.8180771, 0.0))

In [48]:
[x[1] - x[0] for x in (zip(mid.point,curr_rent.point))]

[-0.002997699999994552, 0.2005827999999994, 0.0]

In [105]:
dist_ = {}
for i in tqdm(df['adress'].unique()):
    try:
        dist_[i]=geocode(f'Москва {i}').point
    except:
        dist_[i]=(0,0,0)

100%|██████████| 1834/1834 [15:16<00:00,  2.00it/s]


In [164]:
df['loc']=df['adress'].map(dist_)
df['loc']=df['loc'].apply(lambda x: x[:2])

In [116]:
df['lng'] = df['loc'].apply(lambda x: x[0])
df['lat'] = df['loc'].apply(lambda x: x[1])

In [117]:
df['lng_norm'] = df['loc'].apply(lambda x: x[0] - mid.point[0])
df['lat_norm'] = df['loc'].apply(lambda x: x[1] - mid.point[1])

In [118]:
df['mos_pos'] = df['loc'].progress_apply(lambda x: [x[1] - x[0] for x in (zip(mid.point[:2],x))])

100%|██████████| 11787/11787 [00:00<00:00, 624354.49it/s]


In [119]:
df['metro_dist_bin']=pd.qcut(df['metro_dist'],4,['<5','5-10','10-15','15<'])

In [121]:
df[['lng_norm','lat_norm','rubm2']].corr()['rubm2']

lng_norm    0.010418
lat_norm    0.001889
rubm2       1.000000
Name: rubm2, dtype: float64

In [179]:
df=df.astype('str').apply(pd.to_numeric,errors='ignore')

In [180]:
with sqlite3.connect('data/avito_real_estate.db') as db:
    df.drop('index',axis=1).to_sql(name='moscow_features',con=db,index=False,if_exists='replace')